In [1]:
#%% Import
from mednickdb_pysleep import sleep_features
# Until pyparse has been merged into pysleep, you will need the latest version of pyparse:
# >>pip install git+https://bdyetton@bitbucket.org/mednicklab/mednickdb_pyparse.git
from mednickdb_pyparse import pyparse_utils, parse_scorefile  
import glob
import pandas as pd

C:\Users\bdyet\Anaconda3\lib\site-packages\mednickdb_pyparse\pyparse_utils.py:10: UserWarning: MednickDB API could not be imported. Some functions will not work.
  warnings.warn('MednickDB API could not be imported. Some functions will not work.') #TODO this should not be needed


In [2]:
#%% Find set of sleep files
drug = 'PBOPBO'
working_dir = 'C:/Users/bdyet/Desktop/Test_for_jing/'
edf_filepaths = glob.glob(working_dir + '*.edf')
channels_to_consider = ['F3', 'F4', 'C3', 'C4', 'O1', 'O2', 'P3', 'P4', 'FP1', 
                        'FP2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'FT9', 'FT10', 'Fz', 'Cz', 
                        'CPz', 'PZ', 'POz', 'FPz']

In [23]:
#%% loop through edf's, extract spindles and then features of spindles (per stage)
spindle_features_cont = []
for edf_filepath in edf_filepaths:
    visitid = int(edf_filepath.split('_V')[-1].split('.')[0])
    subjectid = int(edf_filepath.split('_V')[-2].split('_')[-1])
    scoring_filepath = edf_filepath.replace('.edf', '.mat')
    stagemap = pyparse_utils.get_stagemap_by_studyid(scoring_filepath, None)
    stagedata = parse_scorefile.parse_scorefile(scoring_filepath, stagemap)
    print('Detecting Spindles for', edf_filepath)
    spindles = sleep_features.detect_spindles(edf_filepath, algo='Ferrarelli2007', chans_to_consider=channels_to_consider)
    spindles = sleep_features.assign_stage_to_feature_events(spindles, stagedata['epochstage'])
    print('Detected Spindles for', edf_filepath)
    spindle_features = sleep_features.sleep_feature_variables_per_stage(spindles, 
                                                                        stagedata['epochstage'], 
                                                                        stages_to_consider=['stage2', 'sws'], 
                                                                        av_across_channels=False)
    spindle_features['subjectid'] = subjectid
    spindle_features['visitid'] = visitid
    spindle_features_cont.append(spindle_features)
    print('Extracted features of spindles for', edf_filepath)
    

C:\Users\bdyet\GoogleDrive\mednickdb_pysleep\mednickdb_pysleep\sleep_dynamics.py:53: RuntimeWarning: invalid value encountered in true_divide
  second.astype(int)/np.expand_dims(np.sum(second, axis=2), 2)


Detecting Spindles for C:/Users/bdyet/Desktop/Test_for_jing\PSTIM_721_V3.edf


Detected Spindles for C:/Users/bdyet/Desktop/Test_for_jing\PSTIM_721_V3.edf
Extracted features of spindles for C:/Users/bdyet/Desktop/Test_for_jing\PSTIM_721_V3.edf
Detecting Spindles for C:/Users/bdyet/Desktop/Test_for_jing\PSTIM_723_V4.edf


C:\Users\bdyet\GoogleDrive\mednickdb_pysleep\mednickdb_pysleep\sleep_dynamics.py:52: RuntimeWarning: invalid value encountered in true_divide
  first.astype(int)/np.expand_dims(np.sum(first, axis=1), 1), \


Detected Spindles for C:/Users/bdyet/Desktop/Test_for_jing\PSTIM_723_V4.edf
Extracted features of spindles for C:/Users/bdyet/Desktop/Test_for_jing\PSTIM_723_V4.edf


In [26]:
#Combine all spindles into a single dataframe and save to csv
all_spindles_info = pd.concat(spindle_features_cont, axis=0)
all_spindles_info.to_csv('PSTIM_'+drug+'_spindle_data.csv', index=False)

,av_freq_peak,av_peak_uV_in_spindle_band,av_peak_time,av_duration,av_density,chan,stage,description,eventtype
0,12.8069,30.8604,0.240234,0.428711,0.0273973,C3,stage2,spindle,sleep_feature
1,12.8845,30.3064,0.857537,1.36696,0.116438,C4,stage2,spindle,sleep_feature
2,9.6,40.8427,0.199219,0.3125,0.00684932,CPz,stage2,spindle,sleep_feature
3,12.602,37.0551,0.80625,1.26484,0.136986,Cz,stage2,spindle,sleep_feature
4,11.6435,37.4253,0.876154,1.63938,0.60274,F3,stage2,spindle,sleep_feature
5,10.9785,37.6754,0.89502,1.68188,0.328767,F4,stage2,spindle,sleep_feature
6,11.9097,27.3516,0.844301,1.69104,0.582192,F7,stage2,spindle,sleep_feature
7,11.8083,27.6308,0.78446,1.61831,0.691781,F8,stage2,spindle,sleep_feature
8,11.22,27.4432,0.740017,1.34223,0.123288,FP1,stage2,spindle,sleep_feature
9,11.722,30.0267,1.07166,1.81288,0.356164,FP2,stage2,spindle,sleep_feature


'723'